In [1]:
import sys

sys.path.append("..")
from lakeml.vaex import Pipeline
models_path = '../models'

In [38]:
!lakectl repo list

INFO[0000]/home/runner/work/lakeFS/lakeFS/cmd/lakectl/cmd/config/config.go:72 github.com/treeverse/lakefs/cmd/lakectl/cmd/config.ReadConfig() loaded configuration from file                fields.file=/Users/yonatanalexander/.lakectl.yaml file=/Users/yonatanalexander/.lakectl.yaml
+---------------+--------------------------------+------------------+-------------------------------------------------------------------------------------------------+
| REPOSITORY    | CREATION DATE                  | DEFAULT REF NAME | STORAGE NAMESPACE                                                                               |
+---------------+--------------------------------+------------------+-------------------------------------------------------------------------------------------------+
| imdb          | 2021-09-11 13:34:02 +0200 CEST | main             | local:///Users/yonatanalexander/Dropbox/Development_box/xdss-projects/lakeml/data/imdb          |
| iris          | 2021-09-11 13:33:50 +0200 CES

###### Secrets

In [39]:
!lakectl branch list lakefs://iris

INFO[0000]/home/runner/work/lakeFS/lakeFS/cmd/lakectl/cmd/config/config.go:72 github.com/treeverse/lakefs/cmd/lakectl/cmd/config.ReadConfig() loaded configuration from file                fields.file=/Users/yonatanalexander/.lakectl.yaml file=/Users/yonatanalexander/.lakectl.yaml
+--------+------------------------------------------------------------------+
| BRANCH | COMMIT ID                                                        |
+--------+------------------------------------------------------------------+
| main   | 30bb5ee3942cfebe971bb49fd1650ad88cc2e40d79c449f85f3f993deb8f9975 |
+--------+------------------------------------------------------------------+



In [40]:
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient
from dotenv import load_dotenv
import os
import vaex
import json
import sys
from collections import defaultdict
from lakeml.vaex.core.variables import *
import warnings
warnings.filterwarnings('ignore')

sys.path.append("..")
from lakeml.vaex import Pipeline
models_path = '../models'

load_dotenv()


configuration = lakefs_client.Configuration()
configuration.username = os.getenv('LAKEFS_USER')
configuration.password = os.getenv('LAKEFS_PASSWORD')
configuration.host = os.getenv('LAKEFS_HOST')
client= LakeFSClient(configuration)


@vaex.register_dataframe_accessor('lakefs', override=True)
class LakeFSAccessor(object):
    def __init__(self, df):
        self.df = df
        self.client = LakeFSClient(configuration)
        
    def _list_branches(self, repo=None):
        repo = repo or self.df.lakefs.repo
        return self.client.branches.list_branches(repo)

    @property
    def branch(self):
        return self.df.variables.get('lakefs',{}).get('branch')
    
    @property
    def branches(self):
        return self._list_branches()['results']
    
    @property
    def repo(self):
        return self.df.variables.get('lakefs',{}).get('repo')

@vaex.register_dataframe_accessor('viz', override=True)
class FacetAccessor(object):
    def __init__(self, df):
        self.df = df
        
    def dtype(self, expression):
        return self.df.dtypes.loc[expression]

    def get_overview_dictionary(self, selection=None, strings=True, virtual=True, sample=None):
        N = len(self.df)
        if sample is not None:
            if type(sample) not in [int, float]:
                raise ValueError(f"'sample' must be an int or a float, got {type(sample)}")
            if sample < 1:
                sample = int(sample * N)

            if N < sample:
                print(f"WARNING: sample {sample} is bigger the data size {N} - using the full dataset")
                sample = N
            sample = self.df.sample(n=sample)
        else:
            sample = self.df


        columns = defaultdict(dict)

        for feature in sample.get_column_names(strings=strings, virtual=virtual)[:]:
            if sample.is_datetime(feature):
                print("Time series is not implemented, skip...")
                continue
            dtype = str(sample.viz.dtype(feature)) if sample.viz.dtype(feature) != str else 'str'
            columns[feature][DTYPE] = dtype
            dtype = sample.viz.dtype(feature)
            if dtype == bool:
                x = self.df[feature].astype(str)
                count = self.df.count(feature, selection=selection, delay=True)
                columns[feature][MIN] = 1
                columns[feature][MAX] = 1
                columns[feature][MEAN] = 1
                columns[feature][MODE] = 1
                columns[feature][NUM_ZERO] = N - np.count_nonzero(x.values),
                columns[feature][COUNTNA] = x.countmissing()
                columns[feature][NUM_UNIQUE] = x.nunique(selection=selection, delay=True)
                columns[feature][VALUE_COUNTS] = x.value_counts(feature)
                self.df.execute()
                columns[feature][COUNT] = 2
                columns[feature][NON_MISSING_COUNT] = columns[feature][COUNT]
                columns[feature][HISTOGRAM] = self.df.bins(x, [columns[feature][MIN], columns[feature][MAX]], shape=2)
            elif dtype == str or dtype.kind in ['S', 'U']:
                count = sample.count(feature, selection=selection, delay=True)
                columns[feature][MIN] = 1
                columns[feature][MAX] = 1
                columns[feature][MEAN] = 1
                columns[feature][MODE] = 1
                columns[feature][NUM_ZERO] = N - np.count_nonzero(sample[feature].values),
                columns[feature][COUNTNA] = sample[feature].countmissing()
                columns[feature][NUM_UNIQUE] = sample[feature].nunique(selection=selection, delay=True)
                columns[feature][VALUE_COUNTS] = sample[feature].value_counts(feature)
                sample.execute()
                columns[feature][COUNT] = int(count.get())
                columns[feature][NON_MISSING_COUNT] = columns[feature][COUNT]
            elif dtype.kind == 'O':
                # this will also properly count NaN-like objects like NaT
                count_na = sample[feature].isna().astype('int').sum(delay=True)
                sample.execute()
                columns[feature][COUNTNA] = int(count_na.get())
            else:
                is_datetime = sample.is_datetime(feature)
                mean = sample.mean(feature, selection=None, delay=True)
                std = sample.std(feature, selection=None, delay=True)
                minmax = sample.minmax(feature, selection=None, delay=True)
                mode = sample.median_approx(feature, selection=None, delay=True)
                if is_datetime:  # this path tests using isna, which test for nat
                    count_na = sample[feature].isna().astype('int').sum(delay=True)
                else:
                    count = sample.count(feature, selection=None, delay=True)
                sample.execute()
                if is_datetime: #### This to debug
                    mode, count_na, mean, std, minmax = np.ndarray([1]), count_na.get(), mean.get(), std.get(), minmax.get()
                    count = N - int(count_na)
                else:
                    mode, count, mean, std, minmax = mode.get(), count.get(), mean.get(), std.get(), minmax.get()
                    count = int(count)
                min_value, max_value = minmax
                columns[feature][MIN] = int(min_value)
                columns[feature][MAX] = int(max_value)
                columns[feature][MEAN] = float(mean)
                columns[feature][STD] = float(std)
                columns[feature][MODE] = float(mode)
                columns[feature][COUNT] = int(count)
                columns[feature][NON_MISSING_COUNT] = columns[feature][COUNT]
                columns[feature][NUM_ZERO] = 0
                columns[feature][COUNTNA] = sample[feature].countmissing()
                columns[feature][HISTOGRAM] = sample.bins(feature, [columns[feature][MIN], columns[feature][MAX]])
            columns[feature][VALUES] = sample[feature]

        return columns
    
    def overview(self):
        from lakeml.facets import Overview
        return Overview(self.df).show()
    

def lakefs_open(path):
    splits = path.split('/')
    repo = splits[2]
    branch = splits[3]
    filename = splits[4]
    path = f"../data/{repo}/{branch}/{filename}"
    ret = vaex.open(path)
    ret.variables['lakefs'] = {'repo':repo,'branch':branch}
    return ret

def lakefs_commit(pipeline, path, hooks=None):
    splits = path.split('/')
    repo = splits[2]
    branch = splits[3]
    filename = splits[4]
    path = f"../data/{repo}/{branch}/{filename}"
    if hooks is not None:
        for hook in hooks:
            if hook(pipeline) is False:
                raise RuntimeError("invalid pipeline")
    pipeline.save(path)

def lakefs_load(path):
    splits = path.split('/')
    repo = splits[2]
    branch = splits[3]
    filename = splits[4]
    path = f"../data/{repo}/{branch}/{filename}"
    return Pipeline.from_file(path)

def validate_evaluation(pipeline):
    safe = True
    for key in ['accuracy']:
        if key not in pipeline.variables.get('evaluation',{}):
            print(f"{key} is missing")
            safe = False
    return safe

def validate_output(pipeline):
    if not 'prediction' in pipeline.example.column_names:
        print("prediction column is missing")
    return 'prediction' in pipeline.example.column_names        


# df = lakefs_open(path='lakefs://iris/main/iris.parquet')


def inference(pipeline, df):
    df.state_set(pipeline.state)
    return df

# Pipeline.inference = inference

# Demo

## Raw data - how it will arrive in production
* The data is aware of it's repo and branch

In [59]:
#open
df = lakefs_open('lakefs://iris/raw/iris.parquet')
print(f"Repo is {df.lakefs.repo} on branch {df.lakefs.branch}")
print(df.dtypes)
df.head(3)

Repo is iris on branch raw
sepal_length     string
sepal_width     float64
petal_length    float64
petal_width     float64
target            int64
dtype: object


#,sepal_length,sepal_width,petal_length,petal_width,target
0,5.9,3,4.2,1.5,1
1,6.1,3,4.6,1.4,1
2,6.6,2.9,4.6,1.3,1


# Clean (new session)

In [60]:
names = {0:'setosa',1:'versicolor',2:'virginica'}
df.variables['names'] = names
df['target'] = df['target'].map(names)
df['sepal_length'] = df['sepal_length'].astype('float64')
df.head()

#,sepal_length,sepal_width,petal_length,petal_width,target
0,5.9,3,4.2,1.5,versicolor
1,6.1,3,4.6,1.4,versicolor
2,6.6,2.9,4.6,1.3,versicolor
3,6.7,3.3,5.7,2.1,virginica
4,5.5,4.2,1.4,0.2,setosa
5,5.1,3.4,1.5,0.2,setosa
6,6.3,2.3,4.4,1.3,versicolor
7,5,3.5,1.3,0.3,setosa
8,6.7,3.1,5.6,2.4,virginica
9,6,2.2,4,1,versicolor


In [45]:
# cleaning pipeline - start
pipeline = Pipeline.from_dataframe(df)
lakefs_commit(pipeline, 'lakefs://iris-pipeline/main/iris-pipeline.pkl')

In [61]:
pipeline.state

{'virtual_columns': {'target': '_map(__target, map_key_set, map_choices, axis=None)',
  'sepal_length': "astype(__sepal_length, 'float64')",
  'petal_ratio': '(petal_length / petal_width)',
  'sepal_ratio': '(sepal_length / sepal_width)',
  'PCA_0': '(petal_length - 3.694166666666667) * -0.5429564921432712 + (petal_width - 1.1775) * -0.23893056500782683 + (sepal_length - 5.8583333333333325) * -0.22001087589901308 + (sepal_width - 3.100833333333333) * 0.06472808984171316 + (petal_ratio - 4.277271656160816) * 0.7644459468563518 + (sepal_ratio - 1.9305068667088405) * -0.10553822033274038',
  'PCA_1': '(petal_length - 3.694166666666667) * -0.6544720603342852 + (petal_width - 1.1775) * -0.21091193910444778 + (sepal_length - 5.8583333333333325) * -0.3243715508002178 + (sepal_width - 3.100833333333333) * 0.01721217369871767 + (petal_ratio - 4.277271656160816) * -0.6404123164571024 + (sepal_ratio - 1.9305068667088405) * -0.10743081133023796',
  'PCA_2': '(petal_length - 3.694166666666667) * -0

# Modelling (new session)


Make a modeling repo

* It's important to use **raw** data, because that what you will get in production
* It's important to split the data **first** - otherwise you can have data drift and bugs with new data

In [46]:
df = lakefs_open('lakefs://iris/raw/iris.parquet')
train, test  = df.split_random(0.8)
print(train.dtypes)
print(train.head(2))

sepal_length     string
sepal_width     float64
petal_length    float64
petal_width     float64
target            int64
dtype: object
  #    sepal_length    sepal_width    petal_length    petal_width    target
  0             5.8            2.6             4              1.2         1
  1             5.1            3.4             1.5            0.2         0


Taking the cleaning pipeline is equivalent to "pull" from main

In [47]:
# clean 
pipeline = lakefs_load('lakefs://iris-pipeline/main/iris-pipeline.pkl')
train = pipeline.inference(train)
print(train.dtypes)
train.head(2)

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
target           object
dtype: object


#,sepal_length,sepal_width,petal_length,petal_width,target
0,5.8,2.6,4,1.2,versicolor
1,5.1,3.4,1.5,0.2,setosa


## Experiment 

### This cell is a full ML pipeline

In [49]:
from vaex.ml import PCA
from vaex.ml.sklearn import Predictor
from lightgbm import LGBMClassifier

it

features = ['petal_length', 'petal_width', 'sepal_length', 'sepal_width', 'petal_ratio', 'sepal_ratio']
pca = vaex.ml.PCA(features=features, n_components=6)
train = pca.fit_transform(train)

features = train.get_column_names(regex='^PCA')
model = Predictor(model=LGBMClassifier(), features=features, target='target')

model.fit(train)
train = model.transform(train)
train.head(2)


#,sepal_length,sepal_width,petal_length,petal_width,target,petal_ratio,sepal_ratio,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,prediction
0,5.8,2.6,4,1.2,versicolor,3.33333,2.23077,-0.944293,0.377649,-0.493826,-0.224169,0.0320309,0.00213373,versicolor
1,5.1,3.4,1.5,0.2,setosa,7.5,1.5,4.12013,-0.124307,0.0962024,-0.0696526,-0.014432,-0.022012,setosa


In [ ]:
%strong machine
train = model.fit_trnsorm(train)
........

In [ ]:
train = model.fit_trnsorm(train.head(10000))

Can check thevalues all the way:
- petal_ratio
- sepal_ratio
- PCA
- LightGBM

In [50]:
# get pipeline/transformations
message = 'made petal_ratio,sepal_ratio, than pca with 6 components, finaly lightgbm'
print(f"description: {message}")
print()

pipeline = Pipeline.from_dataframe(train)
pipeline.variables['lakefs']['description'] = message
print('Raw data:')
print(test.head(2))
print("\nFinal data")
pipeline.inference(test.head(2))

description: made petal_ratio,sepal_ratio, than pca with 6 components, finaly lightgbm

Raw data:
  #    sepal_length    sepal_width    petal_length    petal_width    target
  0             6.1            2.9             4.7            1.4         1
  1             5.5            2.6             4.4            1.2         1

Final data


#,sepal_length,sepal_width,petal_length,petal_width,target,petal_ratio,sepal_ratio,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,prediction
0,6.1,2.9,4.7,1.4,versicolor,3.35714,2.10345,-1.3871,-0.216382,-0.175183,0.00428362,0.186113,-0.0170203,versicolor
1,5.5,2.6,4.4,1.2,versicolor,3.66667,2.11538,-0.82848,0.0120964,-0.713143,0.124085,0.272866,0.00333408,versicolor


Use hooks to protect yourself like with code

In [51]:
lakefs_commit(pipeline,'lakefs://iris-pipeline/lightgbm/iris-pipeline.pkl', [validate_evaluation, validate_output])

accuracy is missing


RuntimeError: invalid pipeline

In [52]:
from sklearn.metrics import accuracy_score
results = pipeline.inference(test)
pipeline.variables['evaluation'] = {'accuracy':accuracy_score(results['target'].values, results['prediction'].values)}
pipeline.variables['evaluation']

{'accuracy': 0.9}

In [53]:
lakefs_commit(pipeline,'lakefs://iris-pipeline/main/pipeline.pkl',[validate_evaluation,validate_output])

Test?

In [54]:
data = {"sepal_length":"6.5",# original issue
        "sepal_width":3.0,
        "petal_length":5.0,
        "petal_width":2}
print(f"Data: {json.dumps(data)}")
pipeline.inference(data)

Data: {"sepal_length": "6.5", "sepal_width": 3.0, "petal_length": 5.0, "petal_width": 2}


#,sepal_length,sepal_width,petal_length,petal_width,target,petal_ratio,sepal_ratio,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,prediction
0,6.5,3,5,2,setosa,2.5,2.16667,-2.43678,-0.125165,0.105994,0.0662105,-0.220816,-0.010867,virginica


# Missing probability? (New session)

"Pull" data and pipelines

In [55]:
import numpy as np
import pyarrow as pa
import vaex

pipeline = lakefs_load('lakefs://iris-pipeline/main/pipeline.pkl')
df = lakefs_open('lakefs://iris/raw/iris.parquet')
df = pipeline.inference(df)
df.head(2)

#,sepal_length,sepal_width,petal_length,petal_width,target,petal_ratio,sepal_ratio,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,prediction
0,5.9,3,4.2,1.5,versicolor,2.8,1.96667,-1.50051,0.527854,-0.0641189,0.142885,-0.0158682,-0.0221702,versicolor
1,6.1,3,4.6,1.4,versicolor,3.28571,2.03333,-1.37353,-0.095937,-0.0599284,0.0473237,0.173799,-0.0233051,versicolor


In [56]:
# Extract the model (can be prettier)
model = pipeline.get_function_model('sklearn_prediction_function')
names = pipeline.variables['names']
model.model

LGBMClassifier()

In [57]:
@vaex.register_function(on_expression=False)
def proba(*columns):
    data = np.array(columns).T
    probabilities = model.model.predict_proba(data)
    return np.array(probabilities)

df.add_function('proba', proba)
df.func.proba(*tuple([df[col] for col in model.features]))
df['probability'] = df.func.proba(*tuple([df[col] for col in model.features]))
df['probabilities'] = df['probability'].apply(lambda x: {names.get(str(i)):value for i,value in enumerate(x)})
df.head(2)

#,sepal_length,sepal_width,petal_length,petal_width,target,petal_ratio,sepal_ratio,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,prediction,probability,probabilities
0,5.9,3,4.2,1.5,versicolor,2.8,1.96667,-1.50051,0.527854,-0.0641189,0.142885,-0.0158682,-0.0221702,versicolor,"'array([3.58224137e-06, 9.99981749e-01, 1.466834...","""{'setosa': 3.582241366291249e-06, 'versicolor':..."
1,6.1,3,4.6,1.4,versicolor,3.28571,2.03333,-1.37353,-0.095937,-0.0599284,0.0473237,0.173799,-0.0233051,versicolor,"'array([2.28276659e-06, 9.99995231e-01, 2.486132...","""{'setosa': 2.282766586470406e-06, 'versicolor':..."


## Merge to production

In [58]:
pipeline = Pipeline.from_dataframe(df)
lakefs_commit(pipeline,'lakefs://iris-pipeline/main/pipeline.pkl',[validate_evaluation,validate_output])

# Explore branches - Every branch a dashboard

In [7]:
lakefs_open('lakefs://iris/raw/iris.parquet').viz.overview()

<class 'vaex.dataframe.DataFrameLocal'>


# Server

In [25]:
pipeline = Pipeline.from_file('../data/iris-pipeline/main/pipeline.pkl')

# TODO
* Create repo
* ingest files - commit
* clean - commit
* ingest files - commit
* clean - commit
* build transformations - commit
* new transformations - commit
* Build model and save it - commit, trigger evaluation test, model update
* New features - commit
* new model - test fails on commit
* new model - goes to production

# What is commit, merge for pipelines?

In [487]:
test.gl.to_records(0)

{'sepal_length': 7.2,
 'sepal_width': 3.0,
 'petal_length': 5.8,
 'petal_width': 1.6,
 'target': 'virginica',
 'petal_ratio': 3.6249999999999996,
 'sepal_ratio': 2.4,
 'PCA_0': -1.911358715987076,
 'PCA_1': -1.5664874990412434,
 'PCA_2': 0.3493955855443512,
 'PCA_3': -0.4669829234076227,
 'PCA_4': 0.24868924471141962,
 'PCA_5': -0.030992514795244286,
 'prediction': 'versicolor'}

In [1]:
!lakectl config

zsh:1: command not found: lakectl


In [8]:
path = '/Users/yonatanalexander/Dropbox/Development_box/cybear/Po/datasets/ds-black.arrow'

In [9]:
data = vaex.open(path)

In [11]:
data.export_parquet('/Users/yonatanalexander/Dropbox/Development_box/cybear/Po/datasets/ds-black.parquet')